In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

https://www.census.gov/topics/public-sector/voting/data/tables.2010.List_1863097513.html#list-tab-List_1863097513

In [2]:
# read in the Excel file and specify the sheet name
df_votes_2010 = pd.read_excel('Election_Results/vote2010.xls', sheet_name='Table 4a', header=[3])

# display the resulting dataframe
df_votes_2010

Unnamed: 0  Unnamed: 1  Unnamed: 2   
0                                       UNITED STATES    229690.0    210800.0  \
1                                             ALABAMA      3526.0      3420.0   
2                                              ALASKA       498.0       478.0   
3                                             ARIZONA      4831.0      4443.0   
4                                            ARKANSAS      2140.0      2068.0   
5                                          CALIFORNIA     27381.0     22767.0   
6                                            COLORADO      3768.0      3473.0   
7                                         CONNECTICUT      2648.0      2463.0   
8                                            DELAWARE       667.0       634.0   
9                                DISTRICT OF COLUMBIA       489.0       441.0   
10                                            FLORIDA     14227.0     12697.0   
11                                            GEORGIA      7119.0      6574.0   
12                                             HAWAII       965.0       892.0   
13                                              IDAHO      1114.0      1048.0   
14                                           ILLINOIS      9619.0      8780.0   
15                                            INDIANA      4777.0      4638.0   
16                                               IOWA      2278.0      2185.0   
17                                             KANSAS      2059.0      1971.0   
18                                           KENTUCKY      3250.0      3158.0   
19                                          LOUISIANA      3314.0      3236.0   
20                                              MAINE      1034.0      1017.0   
21                                           MARYLAND      4279.0      3853.0   
22                                      MASSACHUSETTS      5097.0      4695.0   
23                                           MICHIGAN      7513.0      7176.0   
24                                          MINNESOTA      3982.0      3830.0   
25                                        MISSISSIPPI      2113.0      2087.0   
26                                           MISSOURI      4506.0      4387.0   
27                                            MONTANA       753.0       744.0   
28                                           NEBRASKA      1323.0      1240.0   
29                                             NEVADA      1957.0      1725.0   
30                                      NEW HAMPSHIRE      1024.0      1002.0   
31                                         NEW JERSEY      6581.0      5719.0   
32                                         NEW MEXICO      1489.0      1362.0   
33                                           NEW YORK     14974.0     13163.0   
34                                     NORTH CAROLINA      6998.0      6632.0   
35                                       NORTH DAKOTA       488.0       477.0   
36                                               OHIO      8642.0      8468.0   
37                                           OKLAHOMA      2695.0      2628.0   
38                                             OREGON      2974.0      2830.0   
39                                       PENNSYLVANIA      9631.0      9339.0   
40                                       RHODE ISLAND       812.0       745.0   
41                                     SOUTH CAROLINA      3453.0      3339.0   
42                                       SOUTH DAKOTA       602.0       587.0   
43                                          TENNESSEE      4745.0      4547.0   
44                                              TEXAS     17847.0     15403.0   
45                                               UTAH      1929.0      1800.0   
46                                            VERMONT       490.0       481.0   
47                                           VIRGINIA      5873.0      5544.0   
48                                         WASHINGTON      5095.0      463

In [3]:
# select the columns wanted and rename them
df_selection = df_votes_2010[['Unnamed: 0', 'Unnamed: 1', 'Total registered', 'Total voted']]
df_votes = df_selection.rename(columns={
    'Unnamed: 0': 'State',
    'Unnamed: 1': 'Population',
    'Total registered': 'Registered',
    'Total voted': 'Voted'
})
# add year and office columns
df_votes['Year'] = 2010

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'State', 'Population', 'Registered', 'Voted']

# df_votes = df_votes.iloc[1:]  # remove the first row
df_votes = df_votes.iloc[:-4] # removes last 7 rows of text

# Save the first row as a separate DataFrame
first_row = df_votes.iloc[[0]].copy()

# Remove the first row from the original DataFrame
df_votes = df_votes.iloc[1:].copy()

# Append the first row to the end of the DataFrame
df_votes = pd.concat([df_votes, first_row])

df_votes = df_votes.reset_index(drop=True)  # reset the index
df_votes = df_votes.reindex(columns=new_cols) # set year column in front

df_votes
# values are in the thousands

Year                 State  Population  Registered    Voted
0   2010               ALABAMA      3526.0      2224.0   1509.0
1   2010                ALASKA       498.0       317.0    242.0
2   2010               ARIZONA      4831.0      2934.0   2170.0
3   2010              ARKANSAS      2140.0      1256.0    841.0
4   2010            CALIFORNIA     27381.0     13864.0  10725.0
5   2010              COLORADO      3768.0      2299.0   1826.0
6   2010           CONNECTICUT      2648.0      1641.0   1215.0
7   2010              DELAWARE       667.0       418.0    323.0
8   2010  DISTRICT OF COLUMBIA       489.0       295.0    200.0
9   2010               FLORIDA     14227.0      7994.0   5572.0
10  2010               GEORGIA      7119.0      4076.0   2865.0
11  2010                HAWAII       965.0       466.0    385.0
12  2010                 IDAHO      1114.0       666.0    498.0
13  2010              ILLINOIS      9619.0      5823.0   3994.0
14  2010               INDIANA      4777.0      2838.0   1826.0
15  2010                  IOWA      2278.0      1547.0   1143.0
16  2010                KANSAS      2059.0      1349.0    931.0
17  2010              KENTUCKY      3250.0      2113.0   1479.0
18  2010             LOUISIANA      3314.0      2425.0   1618.0
19  2010                 MAINE      1034.0       779.0    604.0
20  2010              MARYLAND      4279.0      2509.0   1802.0
21  2010         MASSACHUSETTS      5097.0      3230.0   2452.0
22  2010              MICHIGAN      7513.0      5127.0   3394.0
23  2010             MINNESOTA      3982.0      2756.0   2104.0
24  2010           MISSISSIPPI      2113.0      1532.0    983.0
25  2010              MISSOURI      4506.0      3013.0   2004.0
26  2010               MONTANA       753.0       491.0    386.0
27  2010              NEBRASKA      1323.0       804.0    511.0
28  2010                NEVADA      1957.0       996.0    731.0
29  2010         NEW HAMPSHIRE      1024.0       666.0    470.0
30  2010            NEW JERSEY      6581.0      3656.0   2383.0
31  2010            NEW MEXICO      1489.0       746.0    579.0
32  2010              NEW YORK     14974.0      8395.0   5746.0
33  2010        NORTH CAROLINA      6998.0      4455.0   3009.0
34  2010          NORTH DAKOTA       488.0       361.0    266.0
35  2010                  OHIO      8642.0      5601.0   3824.0
36  2010              OKLAHOMA      2695.0      1603.0   1061.0
37  2010                OREGON      2974.0      2005.0   1594.0
38  2010          PENNSYLVANIA      9631.0      6031.0   4104.0
39  2010          RHODE ISLAND       812.0       510.0    348.0
40  2010        SOUTH CAROLINA      3453.0      2304.0   1700.0
41  2010          SOUTH DAKOTA       602.0       406.0    322.0
42  2010             TENNESSEE      4745.0      2854.0   1714.0
43  2010                 TEXAS     17847.0      9493.0   5600.0
44  2010                  UTAH      1929.0      1095.0    695.0
45  2010               VERMONT       490.0       356.0    265.0
46  2010              VIRGINIA      5873.0      3546.0   2257.0
47  2010            WASHINGTON      5095.0      3367.0   2692.0
48  2010         WEST VIRGINIA      1420.0       883.0    581.0
49  2010             WISCONSIN      4291.0      2908.0   2258.0
50  2010               WYOMING       411.0       239.0    190.0
51  2010         UNITED STATES    229690.0    137263.0  95987.0

In [4]:
state_abbr = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
    'UNITED STATES': 'US'
}



In [5]:
df_votes['State'] = df_votes['State'].map(state_abbr)
df_votes

Year State  Population  Registered    Voted
0   2010    AL      3526.0      2224.0   1509.0
1   2010    AK       498.0       317.0    242.0
2   2010    AZ      4831.0      2934.0   2170.0
3   2010    AR      2140.0      1256.0    841.0
4   2010    CA     27381.0     13864.0  10725.0
5   2010    CO      3768.0      2299.0   1826.0
6   2010    CT      2648.0      1641.0   1215.0
7   2010    DE       667.0       418.0    323.0
8   2010    DC       489.0       295.0    200.0
9   2010    FL     14227.0      7994.0   5572.0
10  2010    GA      7119.0      4076.0   2865.0
11  2010    HI       965.0       466.0    385.0
12  2010    ID      1114.0       666.0    498.0
13  2010    IL      9619.0      5823.0   3994.0
14  2010    IN      4777.0      2838.0   1826.0
15  2010    IA      2278.0      1547.0   1143.0
16  2010    KS      2059.0      1349.0    931.0
17  2010    KY      3250.0      2113.0   1479.0
18  2010    LA      3314.0      2425.0   1618.0
19  2010    ME      1034.0       779.0    604.0
20  2010    MD      4279.0      2509.0   1802.0
21  2010    MA      5097.0      3230.0   2452.0
22  2010    MI      7513.0      5127.0   3394.0
23  2010    MN      3982.0      2756.0   2104.0
24  2010    MS      2113.0      1532.0    983.0
25  2010    MO      4506.0      3013.0   2004.0
26  2010    MT       753.0       491.0    386.0
27  2010    NE      1323.0       804.0    511.0
28  2010    NV      1957.0       996.0    731.0
29  2010    NH      1024.0       666.0    470.0
30  2010    NJ      6581.0      3656.0   2383.0
31  2010    NM      1489.0       746.0    579.0
32  2010    NY     14974.0      8395.0   5746.0
33  2010    NC      6998.0      4455.0   3009.0
34  2010    ND       488.0       361.0    266.0
35  2010    OH      8642.0      5601.0   3824.0
36  2010    OK      2695.0      1603.0   1061.0
37  2010    OR      2974.0      2005.0   1594.0
38  2010    PA      9631.0      6031.0   4104.0
39  2010    RI       812.0       510.0    348.0
40  2010    SC      3453.0      2304.0   1700.0
41  2010    SD       602.0       406.0    322.0
42  2010    TN      4745.0      2854.0   1714.0
43  2010    TX     17847.0      9493.0   5600.0
44  2010    UT      1929.0      1095.0    695.0
45  2010    VT       490.0       356.0    265.0
46  2010    VA      5873.0      3546.0   2257.0
47  2010    WA      5095.0      3367.0   2692.0
48  2010    WV      1420.0       883.0    581.0
49  2010    WI      4291.0      2908.0   2258.0
50  2010    WY       411.0       239.0    190.0
51  2010    US    229690.0    137263.0  95987.0

In [6]:
# Calculate the normalized voter count
df_votes['Norm_Voter_Reg'] = df_votes['Voted'] / df_votes['Registered']
df_votes['Norm_Voter_Pop'] = df_votes['Voted'] / df_votes['Population']
df_votes

Year State  Population  Registered    Voted  Norm_Voter_Reg   
0   2010    AL      3526.0      2224.0   1509.0        0.678507  \
1   2010    AK       498.0       317.0    242.0        0.763407   
2   2010    AZ      4831.0      2934.0   2170.0        0.739605   
3   2010    AR      2140.0      1256.0    841.0        0.669586   
4   2010    CA     27381.0     13864.0  10725.0        0.773586   
5   2010    CO      3768.0      2299.0   1826.0        0.794258   
6   2010    CT      2648.0      1641.0   1215.0        0.740402   
7   2010    DE       667.0       418.0    323.0        0.772727   
8   2010    DC       489.0       295.0    200.0        0.677966   
9   2010    FL     14227.0      7994.0   5572.0        0.697023   
10  2010    GA      7119.0      4076.0   2865.0        0.702895   
11  2010    HI       965.0       466.0    385.0        0.826180   
12  2010    ID      1114.0       666.0    498.0        0.747748   
13  2010    IL      9619.0      5823.0   3994.0        0.685901   
14  2010    IN      4777.0      2838.0   1826.0        0.643411   
15  2010    IA      2278.0      1547.0   1143.0        0.738849   
16  2010    KS      2059.0      1349.0    931.0        0.690141   
17  2010    KY      3250.0      2113.0   1479.0        0.699953   
18  2010    LA      3314.0      2425.0   1618.0        0.667216   
19  2010    ME      1034.0       779.0    604.0        0.775353   
20  2010    MD      4279.0      2509.0   1802.0        0.718214   
21  2010    MA      5097.0      3230.0   2452.0        0.759133   
22  2010    MI      7513.0      5127.0   3394.0        0.661986   
23  2010    MN      3982.0      2756.0   2104.0        0.763425   
24  2010    MS      2113.0      1532.0    983.0        0.641645   
25  2010    MO      4506.0      3013.0   2004.0        0.665118   
26  2010    MT       753.0       491.0    386.0        0.786151   
27  2010    NE      1323.0       804.0    511.0        0.635572   
28  2010    NV      1957.0       996.0    731.0        0.733936   
29  2010    NH      1024.0       666.0    470.0        0.705706   
30  2010    NJ      6581.0      3656.0   2383.0        0.651805   
31  2010    NM      1489.0       746.0    579.0        0.776139   
32  2010    NY     14974.0      8395.0   5746.0        0.684455   
33  2010    NC      6998.0      4455.0   3009.0        0.675421   
34  2010    ND       488.0       361.0    266.0        0.736842   
35  2010    OH      8642.0      5601.0   3824.0        0.682735   
36  2010    OK      2695.0      1603.0   1061.0        0.661884   
37  2010    OR      2974.0      2005.0   1594.0        0.795012   
38  2010    PA      9631.0      6031.0   4104.0        0.680484   
39  2010    RI       812.0       510.0    348.0        0.682353   
40  2010    SC      3453.0      2304.0   1700.0        0.737847   
41  2010    SD       602.0       406.0    322.0        0.793103   
42  2010    TN      4745.0      2854.0   1714.0        0.600561   
43  2010    TX     17847.0      9493.0   5600.0        0.589908   
44  2010    UT      1929.0      1095.0    695.0        0.634703   
45  2010    VT       490.0       356.0    265.0        0.744382   
46  2010    VA      5873.0      3546.0   2257.0        0.636492   
47  2010    WA      5095.0      3367.0   2692.0        0.799525   
48  2010    WV      1420.0       883.0    581.0        0.657984   
49  2010    WI      4291.0      2908.0   2258.0        0.776479   
50  2010    WY       411.0       239.0    190.0        0.794979   
51  2010    US    229690.0    137263.0  95987.0        0.699293   

    Norm_Voter_Pop  
0         0.427964  
1         0.485944  
2         0.449182  
3         0.392991  
4         0.391695  
5         0.484607  
6         0.458837  
7         0.484258  
8         0.408998  
9         0.391650  
10        0.402444  
11        0.398964  
12        0.447038  
13        0.415220  
14        0.382248  
15        0.501756  
16        0.452161  
17        0.455077  
18        0.488232  
19        0.584139  
20        0

# Add to sql db

In [7]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the voters table
df_votes.to_sql("voters", engine, if_exists="append", index=False)
print("Votes table loaded successfully")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


Votes table loaded successfully


In [8]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM voters", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()
# gives 364 rows for 2020 to 2010 and 7 columns

     Year State  Population  Registered  Voted  Norm_Voter_Reg  Norm_Voter_Pop
0    2020    AL      3769.0        2527   2247        0.889197        0.596179
1    2020    AK       528.0         383    330        0.861619        0.625000
2    2020    AZ      5638.0        3878   3649        0.940949        0.647215
3    2020    AR      2283.0        1361   1186        0.871418        0.519492
4    2020    CA     30342.0       18001  16893        0.938448        0.556753
..    ...   ...         ...         ...    ...             ...             ...
255  2010    WA      5095.0        3367   2692        0.799525        0.528361
256  2010    WV      1420.0         883    581        0.657984        0.409155
257  2010    WI      4291.0        2908   2258        0.776479        0.526218
258  2010    WY       411.0         239    190        0.794979        0.462287
259  2010    US    229690.0      137263  95987        0.699293        0.417898

[260 rows x 7 columns]
